In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
def issue_time_lastrow(dbname):
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    c.execute('SELECT * FROM county ORDER BY id DESC LIMIT 1')
    row = c.fetchall()
    if row == []:
        # empty db
        return None
    else:
        return row[0][1]

def parse_data_into_df(data, df):
    for loc in locations:
        loc_name = loc['locationName']
        elements = loc['weatherElement']
        for element in elements:
            element_name = element['elementName']
            for etime in element['time']:
                start_time = etime['startTime']
                end_time = etime['endTime']
                parameter = etime['parameter']['parameterName']
                df = df.append([[issue_time, loc_name, element_name, start_time, end_time, parameter]])

    df = df.reset_index(drop=True)
    df.columns = ['issueTime', 'location', 'element', 'startTime', 'endTime', 'value']
    return df

In [18]:
with open('my_cwb_apikey.txt') as f:
    apikey = f.readline()
format_ = 'json'
dataid = 'F-C0032-001'   # 一般天氣預報-今明36小時天氣預報

url = f'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/{dataid}?Authorization={apikey}&format={format_}'
resp = requests.get(url)
data = resp.json()

In [19]:
issue_time = data['cwbopendata']['dataset']['datasetInfo']['issueTime']
update = data['cwbopendata']['dataset']['datasetInfo']['update']
locations = data['cwbopendata']['dataset']['location']
print(issue_time)
print(update)

2019-08-28T11:00:00+08:00
2019-08-28T11:12:05+08:00


In [20]:
last_issue = issue_time_lastrow('county_36hr_forecast.sqlite')

if not last_issue == issue_time:
    # convert into df
    df = pd.DataFrame()
    df = parse_data_into_df(data, df)
    print(df.head(8))
    
    # append data into db
    conn = sqlite3.connect('county_36hr_forecast.sqlite')
    df.to_sql('county', conn, if_exists='append', index=False)
    conn.close()
else:
    print('data has existed')

last_issue:  2019-08-27T23:00:00+08:00
issu_time:   2019-08-28T11:00:00+08:00
Equal?  False
                   issueTime location element                  startTime  \
0  2019-08-28T11:00:00+08:00      臺北市      Wx  2019-08-28T12:00:00+08:00   
1  2019-08-28T11:00:00+08:00      臺北市      Wx  2019-08-28T18:00:00+08:00   
2  2019-08-28T11:00:00+08:00      臺北市      Wx  2019-08-29T06:00:00+08:00   
3  2019-08-28T11:00:00+08:00      臺北市    MaxT  2019-08-28T12:00:00+08:00   
4  2019-08-28T11:00:00+08:00      臺北市    MaxT  2019-08-28T18:00:00+08:00   
5  2019-08-28T11:00:00+08:00      臺北市    MaxT  2019-08-29T06:00:00+08:00   
6  2019-08-28T11:00:00+08:00      臺北市    MinT  2019-08-28T12:00:00+08:00   
7  2019-08-28T11:00:00+08:00      臺北市    MinT  2019-08-28T18:00:00+08:00   

                     endTime      value  
0  2019-08-28T18:00:00+08:00       晴時多雲  
1  2019-08-29T06:00:00+08:00       晴時多雲  
2  2019-08-29T18:00:00+08:00  多雲午後短暫雷陣雨  
3  2019-08-28T18:00:00+08:00         36  
4  2019-08-29

In [23]:
# read data from db
conn = sqlite3.connect('county_36hr_forecast.sqlite')
c = conn.cursor()
c.execute('SELECT * FROM county')
rows = c.fetchall()
columns = [des[0] for des in c.description]
print(columns)

for row in rows[-100:]:
    print(row)

conn.close()

['id', 'issueTime', 'location', 'element', 'startTime', 'endTime', 'value']
(1, '2019-08-28T11:00:00+08:00', '臺北市', 'Wx', '2019-08-28T12:00:00+08:00', '2019-08-28T18:00:00+08:00', '晴時多雲')
(2, '2019-08-28T11:00:00+08:00', '臺北市', 'Wx', '2019-08-28T18:00:00+08:00', '2019-08-29T06:00:00+08:00', '晴時多雲')
(3, '2019-08-28T11:00:00+08:00', '臺北市', 'Wx', '2019-08-29T06:00:00+08:00', '2019-08-29T18:00:00+08:00', '多雲午後短暫雷陣雨')
(4, '2019-08-28T11:00:00+08:00', '臺北市', 'MaxT', '2019-08-28T12:00:00+08:00', '2019-08-28T18:00:00+08:00', '36')
(5, '2019-08-28T11:00:00+08:00', '臺北市', 'MaxT', '2019-08-28T18:00:00+08:00', '2019-08-29T06:00:00+08:00', '32')
(6, '2019-08-28T11:00:00+08:00', '臺北市', 'MaxT', '2019-08-29T06:00:00+08:00', '2019-08-29T18:00:00+08:00', '35')
(7, '2019-08-28T11:00:00+08:00', '臺北市', 'MinT', '2019-08-28T12:00:00+08:00', '2019-08-28T18:00:00+08:00', '32')
(8, '2019-08-28T11:00:00+08:00', '臺北市', 'MinT', '2019-08-28T18:00:00+08:00', '2019-08-29T06:00:00+08:00', '27')
(9, '2019-08-28T11:00:0